# TESSERA Embeddings Usage

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/tessera.ipynb)

This notebook demonstrates how to use [TESSERA](https://github.com/ucam-eo/tessera) (Temporal Embeddings of Surface Spectra for Earth Representation and Analysis) embeddings through the geoai package.

TESSERA is a foundation model developed at the University of Cambridge that processes time-series Sentinel-1 and Sentinel-2 satellite imagery to generate 128-channel representation maps at 10m resolution globally. These embeddings compress a full year of temporal-spectral features into dense representations suitable for downstream geospatial analysis tasks such as land classification, canopy height prediction, and environmental change detection.

## Installation

Uncomment the following line to install the required packages.

In [ ]:
# %pip install geoai-py geotessera

## Import Library

In [ ]:
import geoai

## Check Available Years

First, let's check which years of TESSERA embeddings are available.

In [ ]:
years = geoai.tessera_available_years()
print(f"Available years: {years}")

## Check Tile Count

Before downloading, check how many tiles are available for a region of interest.

In [ ]:
# Define a bounding box for Cambridge, UK
bbox = (0.05, 52.15, 0.25, 52.25)

count = geoai.tessera_tile_count(bbox=bbox, year=2024)
print(f"{count} tiles available for the specified region")

## Generate Coverage Map

Generate a coverage map to visualize data availability before downloading.

In [ ]:
# Global coverage for 2024
geoai.tessera_coverage(year=2024, output_path="tessera_coverage_2024.png")

In [ ]:
# Regional coverage for Europe
geoai.tessera_coverage(
    year=2024, region_bbox=(-10, 35, 40, 60), output_path="tessera_coverage_europe.png"
)

## Download Embeddings as GeoTIFF

Download TESSERA embeddings for a region. GeoTIFF format includes georeferencing information for direct use in GIS workflows.

In [ ]:
# Download embeddings for Cambridge, UK
bbox = (0.05, 52.15, 0.25, 52.25)

files = geoai.tessera_download(
    bbox=bbox, year=2024, output_dir="./tessera_cambridge", output_format="tiff"
)
print(f"Downloaded {len(files)} files")
for f in files:
    print(f"  {f}")

## Download a Single Tile

You can also download a single tile by specifying a point location.

In [ ]:
files = geoai.tessera_download(
    lon=0.15, lat=52.05, year=2024, output_dir="./tessera_single_tile"
)
print(f"Downloaded {len(files)} file(s)")

## Download Specific Bands

Download only specific bands to reduce file size.

In [ ]:
files = geoai.tessera_download(
    bbox=bbox, year=2024, bands=[0, 1, 2], output_dir="./tessera_rgb_only"
)
print(f"Downloaded {len(files)} files with 3 bands each")

## Visualize Embeddings

Create false-color RGB composites from the embedding bands.

In [ ]:
# Visualize with default bands (0, 1, 2)
geoai.tessera_visualize_rgb(
    "./tessera_cambridge", bands=(0, 1, 2), title="Cambridge - TESSERA Bands 0, 1, 2"
)

In [ ]:
# Try different band combinations
geoai.tessera_visualize_rgb(
    "./tessera_cambridge",
    bands=(30, 60, 90),
    title="Cambridge - TESSERA Bands 30, 60, 90",
)

In [ ]:
# Save visualization to file
geoai.tessera_visualize_rgb(
    "./tessera_cambridge", bands=(0, 1, 2), output_path="tessera_cambridge_rgb.png"
)

## Fetch Embeddings to Memory

For immediate analysis, fetch embeddings directly into numpy arrays without saving to disk.

In [ ]:
tiles = geoai.tessera_fetch_embeddings(bbox=(0.05, 52.15, 0.25, 52.25), year=2024)

for tile in tiles:
    print(f"Tile ({tile['lon']:.2f}, {tile['lat']:.2f}):")
    print(f"  Shape: {tile['embedding'].shape}")
    print(f"  CRS: {tile['crs']}")
    print(f"  Mean: {tile['embedding'].mean():.4f}")
    print(f"  Std: {tile['embedding'].std():.4f}")

## Sample Embeddings at Point Locations

Extract embedding values at specific geographic point locations. This is useful for creating feature vectors for downstream machine learning tasks.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Create sample points
points = gpd.GeoDataFrame(
    {"name": ["Point A", "Point B", "Point C"]},
    geometry=[Point(0.12, 52.20), Point(0.15, 52.18), Point(0.20, 52.22)],
    crs="EPSG:4326",
)

# Sample TESSERA embeddings at these points
result = geoai.tessera_sample_points(points, year=2024)

print(f"Result shape: {result.shape}")
print(
    f"\nOriginal columns: {[c for c in result.columns if not c.startswith('tessera_')]}"
)
print(f"Embedding columns: tessera_0 through tessera_127")
print(f"\nFirst few embedding values for Point A:")
print(result.iloc[0][[f"tessera_{i}" for i in range(5)]])

## Download as NumPy Arrays

For programmatic workflows, download embeddings as raw numpy arrays with metadata.

In [ ]:
import json
import numpy as np

files = geoai.tessera_download(
    bbox=bbox, year=2024, output_dir="./tessera_numpy", output_format="npy"
)

# Load metadata
with open("./tessera_numpy/metadata.json") as f:
    meta = json.load(f)

print(f"Year: {meta['year']}")
print(f"Number of tiles: {len(meta['tiles'])}")
for tile_info in meta["tiles"]:
    arr = np.load(f"./tessera_numpy/{tile_info['file']}")
    print(f"  {tile_info['file']}: shape={arr.shape}, dtype={arr.dtype}")

## Download with Region File

Use a GeoJSON or Shapefile to define the download region.

In [ ]:
# Create a sample region file
import geopandas as gpd
from shapely.geometry import box

region = gpd.GeoDataFrame(geometry=[box(0.05, 52.15, 0.25, 52.25)], crs="EPSG:4326")
region.to_file("cambridge_region.geojson", driver="GeoJSON")

# Download using the region file
files = geoai.tessera_download(
    region_file="cambridge_region.geojson",
    year=2024,
    output_dir="./tessera_from_region",
)
print(f"Downloaded {len(files)} files")